In [ ]:
'''
import zipfile
import os

# Define the path to your zip file
zip_file_path = '/content/files.zip'

# Define the directory where you want to extract the contents
extract_to_dir = '/content/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_to_dir, exist_ok=True)

try:
    # Open the zip file in read mode
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Extract all contents to the specified directory
        zip_ref.extractall(extract_to_dir)
    print(f"Successfully extracted '{zip_file_path}' to '{extract_to_dir}'")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
except FileNotFoundError:
    print(f"Error: Zip file not found at '{zip_file_path}'.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
  '''


'\nimport zipfile\nimport os\n\n# Define the path to your zip file\nzip_file_path = \'/content/files.zip\'\n\n# Define the directory where you want to extract the contents\nextract_to_dir = \'/content/\'\n\n# Create the extraction directory if it doesn\'t exist\nos.makedirs(extract_to_dir, exist_ok=True)\n\ntry:\n    # Open the zip file in read mode\n    with zipfile.ZipFile(zip_file_path, \'r\') as zip_ref:\n        # Extract all contents to the specified directory\n        zip_ref.extractall(extract_to_dir)\n    print(f"Successfully extracted \'{zip_file_path}\' to \'{extract_to_dir}\'")\nexcept zipfile.BadZipFile:\n    print(f"Error: \'{zip_file_path}\' is not a valid zip file.")\nexcept FileNotFoundError:\n    print(f"Error: Zip file not found at \'{zip_file_path}\'.")\nexcept Exception as e:\n    print(f"An unexpected error occurred: {e}")\n  '

In [ ]:
import pandas as pd

insert_len = 3
can_read = False
read_counter = 0
parsed_dataframes = []
'''
table_structure
  `id` bigint NOT NULL AUTO_INCREMENT,
  `designite_id` bigint DEFAULT NULL,
  `has_smell` bit(1) NOT NULL,
  `is_class` bit(1) NOT NULL,
  `path_to_file` varchar(255) DEFAULT NULL,
  `project_name` varchar(255) DEFAULT NULL,
  `sample_constraints` int NOT NULL,
  `smells` varchar(255) DEFAULT NULL
'''

'''
sample = pd.DataFrame({
    "has_smell": dtype=bool,
    "is_class": dtype=bool,
    "project_name": dtype=string,
    "path_to_file": dtype=string,
    'smells': dtype=int
})
'''

with open("/content/DACOSExtended.sql", 'r') as file:
  for line in file:
    if(line.strip() == "/*!40000 ALTER TABLE `sample` DISABLE KEYS */;"):
      can_read = True
      continue

    if(line.strip() == "/*!40000 ALTER TABLE `sample` ENABLE KEYS */;"):
      break

    if(can_read):
      read_counter += 1
      values = line.strip()[len("INSERT INTO `sample` VALUES ("): ].split("),(")
      values = values[:-1]
      '''
      for data in values:
        print(data.split(","))
      '''
      values = [x.replace('"', '').replace("'", '').split(",") for x in values]

      df = None
      try:
        df = pd.DataFrame(values, columns=["id", "designite_id", "has_smell", "is_class", "path_to_file", "project_name", "sample_constraints", "smell"])
      except ValueError:
        df = pd.DataFrame(values, columns=["id", "designite_id", "has_smell", "is_class", "path_to_file", "project_name", "sample_constraints", "smell 1", "smell 2"])

      parsed_dataframes.append(df)

len(parsed_dataframes)

20

In [ ]:
merged_dataframe = pd.concat(parsed_dataframes)

merged_dataframe.loc[merged_dataframe['is_class'] != "_binary ", 'path_to_file']
merged_dataframe

,id,designite_id,has_smell,is_class,path_to_file,project_name,sample_constraints,smell 1,smell 2,smell
0,1,370,_binary \0,_binary ,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,0,,None,NaN
1,2,371,_binary \0,_binary ,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,0,,None,NaN
2,3,379,_binary ,_binary ,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,2,3,None,NaN
3,4,7,_binary \0,_binary ,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,0,,None,NaN
4,5,8,_binary \0,_binary ,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,0,,None,NaN
...,...,...,...,...,...,...,...,...,...,...
1338,109999,177266,_binary \0,_binary \0,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,0,,None,NaN
1339,110000,177268,_binary \0,_binary \0,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,0,,None,NaN
1340,110001,177269,_binary \0,_binary \0,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,0,,None,NaN
1341,110002,177270,_binary \0,_binary \0,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,0,,None,NaN


In [ ]:
import numpy as np


clean = merged_dataframe.drop(columns=['id', "designite_id", "has_smell", "sample_constraints", "project_name"])

clean.fillna("", inplace=True)
clean['smell_type'] = clean['smell 1'] + ", " + clean['smell 2']
clean.drop(columns=['smell 2', 'smell 1'], inplace=True)

clean['smell_type'] = clean['smell_type'].str.replace(',', "")
clean['smell_type'] = clean['smell_type'].replace(r'^\s*$|^nan$|^None$', None, regex=True)
clean['is_smelly'] = clean['smell_type'].notna()

clean['name'] = clean['path_to_file'].str.split('/').str[-1].str[:-5]

clean["type"] = np.where(clean["is_class"] != "_binary ", "Method", "Class")
clean.drop(columns=['is_class'], inplace=True)

clean['Project'] = clean['path_to_file'].str.split('/').str[2]

clean

,path_to_file,smell,smell_type,is_smelly,name,type,Project
0,/codesplit_java_class/Blankj_AndroidUtilCode/B...,,None,False,OnBackListener,Class,Blankj_AndroidUtilCode
1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,,None,False,OnRefreshListener,Class,Blankj_AndroidUtilCode
2,/codesplit_java_class/Blankj_AndroidUtilCode/B...,,3,True,ClearCacheDebug,Class,Blankj_AndroidUtilCode
3,/codesplit_java_class/Blankj_AndroidUtilCode/B...,,None,False,UtilCodeApi,Class,Blankj_AndroidUtilCode
4,/codesplit_java_class/Blankj_AndroidUtilCode/B...,,None,False,Callback,Class,Blankj_AndroidUtilCode
...,...,...,...,...,...,...,...
1338,/codesplit_java_method/MovingBlocks_Terasology...,,None,False,put,Method,MovingBlocks_Terasology
1339,/codesplit_java_method/MovingBlocks_Terasology...,,None,False,toString,Method,MovingBlocks_Terasology
1340,/codesplit_java_method/MovingBlocks_Terasology...,,None,False,get,Method,MovingBlocks_Terasology
1341,/codesplit_java_method/MovingBlocks_Terasology...,,None,False,put,Method,MovingBlocks_Terasology


In [ ]:
clean['path_to_file'] = clean['path_to_file'].str.replace("\\", "/")

clean = clean[['path_to_file', 'Project', 'type', 'name', "is_smelly", "smell_type"]]
clean

,path_to_file,Project,type,name,is_smelly,smell_type
0,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnBackListener,False,None
1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnRefreshListener,False,None
2,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,ClearCacheDebug,True,3
3,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,UtilCodeApi,False,None
4,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,Callback,False,None
...,...,...,...,...,...,...
1338,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None
1339,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,toString,False,None
1340,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,get,False,None
1341,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None


Checking the data for any duplicates and null entries, preforming cleaning if necessary

In [ ]:
clean.notnull().describe()

,path_to_file,Project,type,name,is_smelly,smell_type
count,99367,99367,99367,99367,99367,99367
unique,1,1,1,1,1,2
top,True,True,True,True,True,False
freq,99367,99367,99367,99367,99367,94451


In [ ]:
clean.duplicated(keep="first").value_counts()

,count
False,96280
True,3087


In [ ]:
counter = 0;

for path in clean[clean.duplicated(keep=False, subset=['path_to_file', 'name'])]['path_to_file'].unique():
  print(clean.loc[clean['path_to_file'] == path, 'path_to_file'].str[-40:])
  counter += 1

  if(counter == 30):
    break

889    .recovery.impl/FileSystemRepository.code
65     .recovery.impl/FileSystemRepository.code
Name: path_to_file, dtype: object
890    xa.recovery.impl/InMemoryRepository.code
66     xa.recovery.impl/InMemoryRepository.code
Name: path_to_file, dtype: object
891    der.zkprocess.zookeeper/DiretoryInf.code
67     der.zkprocess.zookeeper/DiretoryInf.code
Name: path_to_file, dtype: object
892    .loader.zkprocess.zookeeper/DataInf.code
68     .loader.zkprocess.zookeeper/DataInf.code
Name: path_to_file, dtype: object
893    der.zkprocess.zookeeper/ClusterInfo.code
69     der.zkprocess.zookeeper/ClusterInfo.code
Name: path_to_file, dtype: object
894    t.config.loader.xml/XMLSchemaLoader.code
70     t.config.loader.xml/XMLSchemaLoader.code
Name: path_to_file, dtype: object
895    t.config.loader.xml/XMLConfigLoader.code
71     t.config.loader.xml/XMLConfigLoader.code
Name: path_to_file, dtype: object
896    cat.config.loader.xml/XMLRuleLoader.code
72     cat.config.loader.xml/XMLRuleLoader

In [ ]:
clean["smell_type"].unique();

In [ ]:
labled_data = clean.drop_duplicates(keep='first', subset=['path_to_file', 'name'], ignore_index=True)
labled_data

,path_to_file,Project,type,name,is_smelly,smell_type
0,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnBackListener,False,None
1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnRefreshListener,False,None
2,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,ClearCacheDebug,True,3
3,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,UtilCodeApi,False,None
4,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,Callback,False,None
...,...,...,...,...,...,...
96275,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None
96276,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,toString,False,None
96277,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,get,False,None
96278,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None


Replacing smell type entries with words for better analysis

In [ ]:
labled_data['smell_type'].value_counts()

,count
smell_type,
1,2098
2,1274
3,1009
1 2,203


In [ ]:
labled_data['smell_type'] = labled_data['smell_type'].astype(str).str.strip()
labled_data.loc[labled_data['smell_type'] == "3", 'smell_type'] = "Complex Method"
labled_data.loc[labled_data['smell_type'] == "2", 'smell_type'] = "Long Parameter List"
labled_data.loc[labled_data['smell_type'] == "1", 'smell_type'] = "Multifaceted Abstraction"
labled_data.loc[labled_data['smell_type'] == "1 2", 'smell_type'] = "Multifaceted Abstraction, Long Parameter List"

labled_data['smell_type'].value_counts()

/tmp/ipython-input-2830520876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labled_data['smell_type'] = labled_data['smell_type'].astype(str).str.strip()


,count
smell_type,
None,91696
Multifaceted Abstraction,2098
Long Parameter List,1274
Complex Method,1009
"Multifaceted Abstraction, Long Parameter List",203


Ensuring no duplicates with this change

In [ ]:
labled_data[labled_data.duplicated(keep=False)].value_counts()

,,,,,,count
path_to_file,Project,type,name,is_smelly,smell_type,


Revising approach to project, making small samples instead of trying to uplaod the entire project or file. Most LLMs cannot handle these massive file sizes

In [ ]:
Label_Data_Classes = labled_data.loc[labled_data['type'] == "Class", :]
Label_Data_Classes

,path_to_file,Project,type,name,is_smelly,smell_type
0,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnBackListener,False,None
1,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,OnRefreshListener,False,None
2,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,ClearCacheDebug,True,Complex Method
3,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,UtilCodeApi,False,None
4,/codesplit_java_class/Blankj_AndroidUtilCode/B...,Blankj_AndroidUtilCode,Class,Callback,False,None
...,...,...,...,...,...,...
59261,/codesplit_java_class/lettuce-io_lettuce-core/...,lettuce-io_lettuce-core,Class,PubSubCommandResp2Test,False,None
59262,/codesplit_java_class/lettuce-io_lettuce-core/...,lettuce-io_lettuce-core,Class,PubSubEndpoint,False,None
59263,/codesplit_java_class/lettuce-io_lettuce-core/...,lettuce-io_lettuce-core,Class,PubSubOutput,False,None
59264,/codesplit_java_class/lettuce-io_lettuce-core/...,lettuce-io_lettuce-core,Class,PubSubCommandHandlerUnitTests,False,None


In [ ]:
print(Label_Data_Classes['is_smelly'].value_counts())

((Label_Data_Classes['is_smelly'].value_counts()[1]/len(Label_Data_Classes)) * 100)

is_smelly
False    7766
True     1009
Name: count, dtype: int64


/tmp/ipython-input-2554951374.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ((Label_Data_Classes['is_smelly'].value_counts()[1]/len(Label_Data_Classes)) * 100)


np.float64(11.498575498575498)

In [ ]:
Label_Data_Methods = labled_data.loc[labled_data['type'] == "Method", :]
Label_Data_Methods

,path_to_file,Project,type,name,is_smelly,smell_type
2320,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,Method,onBack,False,None
2321,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,Method,onRefresh,False,None
2322,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,Method,onSuccess,False,None
2323,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,Method,onClick,False,None
2324,/codesplit_java_method/Blankj_AndroidUtilCode/...,Blankj_AndroidUtilCode,Method,onAppCreate,False,None
...,...,...,...,...,...,...
96275,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None
96276,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,toString,False,None
96277,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,get,False,None
96278,/codesplit_java_method/MovingBlocks_Terasology...,MovingBlocks_Terasology,Method,put,False,None


In [ ]:
print(Label_Data_Methods['is_smelly'].value_counts())

(Label_Data_Methods['is_smelly'].value_counts()[1]/len(Label_Data_Methods)) * 100

is_smelly
False    83930
True      3575
Name: count, dtype: int64


/tmp/ipython-input-1647843624.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  (Label_Data_Methods['is_smelly'].value_counts()[1]/len(Label_Data_Methods)) * 100


np.float64(4.085480829666876)

In [ ]:
projects = {}

for proj in labled_data['Project'].unique():
  projects[proj] = (Label_Data_Classes[Label_Data_Classes['Project'] == proj], Label_Data_Methods[Label_Data_Methods['Project'] == proj])
  print(f"{proj}: \n{projects[proj][0]["is_smelly"].value_counts()} {projects[proj][1]["is_smelly"].value_counts()}\n")

Blankj_AndroidUtilCode: 
is_smelly
False    308
True      96
Name: count, dtype: int64 is_smelly
False    3530
True      282
Name: count, dtype: int64

macrozheng_mall: 
is_smelly
False    487
True      34
Name: count, dtype: int64 is_smelly
False    17357
True        58
Name: count, dtype: int64

linlinjava_litemall: 
is_smelly
False    267
True      77
Name: count, dtype: int64 is_smelly
False    13925
True       168
Name: count, dtype: int64

shuzheng_zheng: 
is_smelly
False    350
True      34
Name: count, dtype: int64 is_smelly
False    6908
True       52
Name: count, dtype: int64

alibaba_arthas: 
is_smelly
False    531
True     136
Name: count, dtype: int64 is_smelly
False    4268
True      394
Name: count, dtype: int64

MyCATApache_Mycat-Server: 
is_smelly
False    1049
True       92
Name: count, dtype: int64 is_smelly
False    6368
True      736
Name: count, dtype: int64

MovingBlocks_Terasology: 
is_smelly
False    2158
True      240
Name: count, dtype: int64 is_smelly
False 

In [ ]:
labled_data.to_csv("Labled_Data.csv", index=False)